In [4]:
from uszipcode import SearchEngine
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import numpy as np


# read input file, rename column containing zip code to ZipCode if necessary
df = pd.read_csv(r"C:\Users\email\OneDrive\Documents\Python\zip_codes\cluster_data_pregrouping.csv")

se = SearchEngine()
zip_df = []
bad_zip = []
for i in df["ZipCode"].dropna().unique():
    row = [i]
    x = se.by_zipcode(i) # x is a SimpleZipCode object
    # appending zip code information:
    # geography:
    row.append(x.lat)
    row.append(x.lng)
    row.append(x.bounds_north)
    row.append(x.bounds_south)
    row.append(x.bounds_east)
    row.append(x.bounds_west)
    row.append(x.radius_in_miles)
    row.append(x.land_area_in_sqmi)
    row.append(x.water_area_in_sqmi)
    # population:
    row.append(x.population)
    row.append(x.population_density)
    # housing:
    row.append(x.housing_units)
    row.append(x.occupied_housing_units)
    row.append(x.median_home_value)
    row.append(x.median_household_income)
    skip = False
    for j in row:
        if j is None:
            # add ZIPs with missing info to this list to impute them
            bad_zip.append(row)
            skip = True
            break
    if not skip: 
        zip_df.append(row)

# converting data into a dataframe:
zipped = pd.DataFrame(zip_df, columns=[
    "ZipCode",      "Latitude",             "Longitude",       "BoundNorth",
    "BoundSouth",   "BoundEast",            "BoundWest",       "RadiusInMiles", 
    "LandArea",     "WaterArea",            "Population",      "PopulationDensity", 
    "HousingUnits", "OccupiedHousingUnits", "MedianHomeValue", "MedianHouseholdIncome" 
    ])

# for bad zips, give them values of their closest neighbor


    

In [6]:
train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
1,0.0,01001,42.06,-72.61,42.100467,42.030795,-72.582535,-72.667902,3.0,11.44,0.86,16769,1466.0,7557,7215,213000,58733
2,1.0,01002,42.37,-72.52,42.437947,42.301437,-72.355041,-72.546776,7.0,55.04,1.65,29049,528.0,10388,9910,338900,54422
3,2.0,01005,42.42,-72.1,42.484473,42.356423,-72.007388,-72.205174,7.0,44.24,0.26,5079,115.0,2044,1904,208500,68644
4,3.0,01007,42.27,-72.4,42.358762,42.185812,-72.331642,-72.472287,8.0,52.64,2.68,14649,278.0,5839,5595,260000,71875
5,4.0,01008,42.18,-72.93,42.25134,42.113028,-72.87218,-73.034916,7.0,53.8,1.96,1263,23.0,586,503,247200,71635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31386,31385.0,99922,55.2,-132.81,55.38444,55.193795,-132.800635,-133.283779,64.0,114.92,8.07,384,3.0,140,129,118100,37361
31387,31386.0,99925,55.54,-132.96,55.638833,55.467957,-132.832076,-133.128372,1.0,55.79,14.25,819,15.0,400,324,167400,38125
31388,31387.0,99926,55.14,-131.49,55.300967,54.986188,-131.293888,-131.667861,26.0,132.8,82.37,1460,11.0,544,509,112000,50052
31389,31388.0,99927,56.3,-133.57,56.358808,56.154967,-133.064024,-133.683215,0.45454545454545453,227.68,6.95,94,0.0,101,55,120000,19000


In [7]:
# enter coordinates into model
train = pd.read_csv(r"C:\Users\email\OneDrive\Documents\Python\zip_codes\zip_reference.csv", header=None).iloc[1:,1:] 
nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(train.iloc[:,1:3])
for i in range(len(bad_zip)):
    # search for closest coordinates
    distances, indices = nbrs.kneighbors(np.reshape(bad_zip[i][1:3], (1, -1)))
    # give bad zip value of closest coordinate
    bad_zip[i][3:] = train.iloc[indices[0,0],3:] 

imputed_zips = pd.DataFrame(bad_zip, columns=[
    "ZipCode",      "Latitude",             "Longitude",       "BoundNorth",
    "BoundSouth",   "BoundEast",            "BoundWest",       "RadiusInMiles", 
    "LandArea",     "WaterArea",            "Population",      "PopulationDensity", 
    "HousingUnits", "OccupiedHousingUnits", "MedianHomeValue", "MedianHouseholdIncome" 
        ])